**计算多尺度自适应插值算子**

In [3]:
%reset -f

In [2]:
import sys, os
sys.path.append(os.path.dirname(os.path.realpath('../utils')))
import numpy as np
from utils.aiwt import cjkfun, z_g
from utils.iwavelets import xphi0

In [ ]:
j0 = 0
J = 15
a = 0
b = 2
V0 = np.sin(np.pi * np.arange(a, b+((b-a)/(2**J)), ((b-a)/(2**J)))).reshape(1,-1)
window = 3
eps0 = 0.005

M = 1
CC = 1

cjk, cjknum, zJc, zJcnum = cjkfun(V0, J, j0, a, b, eps0)
gjk, gjknum, gJc, gJcnum = z_g(j0, J, cjk, cjknum, zJc, zJcnum, a, b, M, CC)

In [ ]:
def restrain(j1, k1, j2, k2):
    """
    输入：
        j1 : int -- 需计算掩码的层
        k1 : ndarray(n,) -- 用于获取配点位置的中间量索引
        j2 : int -- 总层数
        k2 : list[] -- 所有配点的位置
    功能：
        限制算子，获取配点各自所在位置的掩码（唯一True）
    输出：
        value : list[ndarray(len(k2), len(k1))], j层配点各自所在位置的掩码
    """
    len1 = k1.shape[0]
    len2 = k2.shape[0]
    j1_j2 = j1 - j2
    t = 2 ** j1_j2
    if j1_j2 <= 0:
        rt = k1 / t
        rt = rt.reshape((-1,1))
        k2 = k2.reshape((1,-1))
        i = np.ones((1,len2))
        rt = rt * i
        i = np.ones((len1,1))
        k2 = i * k2
        value = ~np.array(abs(rt - k2), dtype=bool)
    return value

In [ ]:
def adapt_CjJ4(J,j0,cjk,cjknum,zJc,zJcnum,a,b):
    """
    输入：
        J : int -- 小波插值层数
        j0 : int -- 初始层数
        cjk : list[ndarray(-1), ...] -- j层每层配点位置索引
        cjknum : list -- 层配点索引数量（每层需计算的配点数量）
        zJc : list -- 配点实际位置
        zJcnum : int -- 实际位置的数量
        a, b : int -- 函数的限制范围
    功能：
        存储每层插值算子C
    输出：
        C : list[ndarray()] -- 所有层的插值算子C
    """
    C = []
    for j in range(j0, J):
        if j > j0:
            if j+1 == len(cjknum):
                break
            elif cjknum[j+1] == 0:
                break
        
        # j层的所有k索引的单变量，向量形式构建，一次性全部算出
        deltaj = (b-a) / (2 ** (j+2))
        k = cjk[j+1].reshape((1,-1))

        # 这一层所有的x_{j+1, 2k+1}
        x = (2 * k + 1) * deltaj

        # 各单变量独自的掩码
        n = zJc.copy()
        rr = restrain(j+2,2 * k + 1,J,n)

        s = 0
        for j1 in range(j0, j+1): # \sum_{j1=j0}^{j-1}
            if cjknum[j1] == 0:
                continue
            # j1层的所有k1索引的单变量，向量形式构建，一次性全部算出
            k1 = cjk[j1].reshape((1,-1))

            # \psi_{j_1 k_1}(x_{j+1,2k+1}) == \phi_{j_1 + 1, 2*k_1 + 1}(x_{j+1,2k+1})
            aa = xphi0(j1+1, 2 * k1 + 1, a, b, x)
            aa = aa.T
            # 在j层的所有k索引的单变量 位置上 堆叠起来的 所有k1索引的单变量
            # 即 \sum_{n=0}^{2^J}（列） \sum_{k_1=0}^{2^{j_1-1}}（行） 在 sum之前 的所有单值
            s = s + np.dot(C[j1-1].T, aa)
        # 计算得到该层C，存储
        C.append(rr - s) if isinstance(s, int) else C.append(rr - s.T)
    return C

In [ ]:
C = adapt_CjJ4(J,j0,gjk,gjknum,gJc,gJcnum,a,b)

In [ ]:
aa = C[5]